# Preparación Datos

In [43]:
# Data handling
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_columns', None)


import glob
import json

# Data visualization
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import missingno as mso
import matplotlib.ticker as ticker
import geopandas as gpd

import ydata_profiling as pp
from sklearn import cluster, mixture
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import kneighbors_graph
from itertools import cycle, islice

import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px

# Stats
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro, norm
# warnings
import warnings
warnings.filterwarnings('ignore')


In [44]:
# Lee el archivo CSV en un DataFrame
df = pd.read_csv('dataset_completo.csv')

In [45]:
# Convierte la columna que contiene la fecha en un formato de fecha
df['Date'] = pd.to_datetime(df['DataAccadimento'])

# Extrae el año, mes, día de la semana y día del mes de la fecha
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day_of_week'] = df['Date'].dt.day_name()
df['Day_of_month'] = df['Date'].dt.day
df.head()

,DataRilevazione,DataProtocollo,DataAccadimento,DataDefinizione,DataMorte,LuogoAccadimento,IdentificativoInfortunato,Genere,Eta,LuogoNascita,ModalitaAccadimento,ConSenzaMezzoTrasporto,IdentificativoCaso,DefinizioneAmministrativa,DefinizioneAmministrativaEsitoMortale,Indennizzo,DecisioneIstruttoriaEsitoMortale,GradoMenomazione,GiorniIndennizzati,IdentificativoDatoreLavoro,PosizioneAssicurativaTerritoriale,SettoreAttivitaEconomica,Gestione,GestioneTariffaria,GrandeGruppoTariffario,Date,Year,Month,Day_of_week,Day_of_month
0,30/04/2023,18/06/2018,17/06/2018,28/06/2018,NaN,69,3895603,M,41,ITAL,N,N,21619890,P,ND,TE,ND,-1,10,9270978,11500123,H 52,I,1,9,2018-06-17,2018,6,Sunday,17
1,30/04/2023,04/10/2021,30/09/2021,27/10/2021,NaN,66,3882671,M,61,ITAL,N,N,23839687,N,ND,NE,ND,-1,0,-1,-1,ND,A,ND,ND,2021-09-30,2021,9,Thursday,30
2,30/04/2023,28/09/2020,23/09/2020,14/11/2020,NaN,66,3897622,M,52,ITAL,N,N,23119539,P,ND,TE,ND,-1,45,3325574,4350088,H 49,I,1,9,2020-09-23,2020,9,Wednesday,23
3,30/04/2023,06/04/2022,04/04/2022,22/04/2022,NaN,69,3896347,F,49,ITAL,N,N,24228399,P,ND,TE,ND,-1,12,4173909,13125854,Q 86,I,4,0,2022-04-04,2022,4,Monday,4
4,30/04/2023,21/02/2020,19/02/2020,17/03/2020,NaN,68,3894028,M,69,ITAL,N,N,22815467,P,ND,TE,ND,-1,26,-1,-1,ND,A,ND,ND,2020-02-19,2020,2,Wednesday,19


In [46]:
df.columns = ['FechaEncuesta', 'FechaRegistro', 'FechaAccidente', 'FechaDefiniciónAdmin', 'FechaMuerte', 'Provincia', 'CódigoInterno',
              'Género', 'Edad', 'LugarNacimiento', 'ModalidadAccidente', 'ImplicaciónMedioTransporte', 'IdentificativoCaso', 'SituaciónAdmin',
              'DefAdminEvidienciaMortal', 'Indemnización', 'DecisiónInstrucciónResultadoMortal', 'GradoDeterioro', 'DíasIndemnizados', 
              'IdentificadorEmpleador', 'PosiciónAseguramientoTerritorial', 'DetalleActividadEconómica', 'Gestión', 'TipoActividadEconómica', 
              'SectorEconómico', 'Fecha', 'Año', 'Mes', 'DiaSemana', 'DiaMes']

In [47]:
conteo_valores = df['GradoDeterioro'].value_counts()

# Imprimir el resultado
print('Número de veces que aparece cada valor único en GradoDeterioro:')
print(conteo_valores)

Número de veces que aparece cada valor único en GradoDeterioro:
GradoDeterioro
-1     2802570
 2       63394
 3       47000
 6       43615
 4       39823
        ...   
 98          2
 76          2
 81          2
 99          1
 83          1
Name: count, Length: 99, dtype: int64


#Aparecen muchas veces el valor -1 en la variable GradoDeterioro. Vamos a eliminar estos valores ya que no aportan informción.

In [48]:
df = df[df['GradoDeterioro'] != -1]
df.shape

(326877, 30)

In [49]:
conteo_edad_menos_uno = df['Edad'].value_counts().get(-1, 0)

print(f"La cantidad de veces que 'Edad' es igual a -1 es: {conteo_edad_menos_uno}")

La cantidad de veces que 'Edad' es igual a -1 es: 11


In [50]:
df = df[df['Edad'] >= 16]

In [51]:
media_grado_deterioro = df['GradoDeterioro'].mean()

print(f"La media de 'GradoDeterioro' es: {media_grado_deterioro}")

La media de 'GradoDeterioro' es: 6.707736071430775


In [52]:
territorial_codes = pd.read_csv('Codici-statistici-e-denominazioni.csv', sep=";", encoding = "ISO-8859-1")


In [53]:
territorial_codes.columns = ['region code', 'Code of the supra-municipal territorial unit', 'Province Code (Historical)', 'Municipality progressive',
                            'Municipality code in alphanumeric format', 'Name (Italian and foreign)', 'Name in Italian', 'Name in other language',
                            'Code Geographical Distribution', 'Geographical distribution', 'Nombre Región', 'Nombre Unidad Territorial',
                            'Type of supra-municipal territorial unit', 'Flag Provincial capital / metropolitan city / free consortium',
                            'Automotive abbreviation', 'Municipal code numeric format', 'Numeric municipality code with 110 provinces (from 2010 to 2016)',
                            'Numeric municipality code with 107 provinces (from 2006 to 2009)', 'Numeric municipality code with 103 provinces (1995 to 2005)',
                            'Cadastral code of the municipality', 'NUTS1 code 2010', 'NUTS2 code 2010', 'NUTS3 code 2010', 'NUTS1 code 2021', 'NUTS2 code 2021',
                            'NUTS3 code 2021']

In [54]:
code_province_name_region = pd.concat([territorial_codes['Province Code (Historical)'],
                                     territorial_codes['Automotive abbreviation'],
                                     territorial_codes['Nombre Región'],
                                      territorial_codes['Nombre Unidad Territorial']], axis = 1)

In [55]:
# When the province is Naples, the abbreviation is NA. This is read as a missing value, replace NA with 'NAPOLI' in full:
code_province_name_region['Automotive abbreviation'][code_province_name_region['Automotive abbreviation'].isna()] = 'NAPOLI' 
code_province_name_region.columns = ['PlaceOfOccurrence', 'abbreviation', 'Region', 'Unidad Territorial' ]

code_province_name_region = code_province_name_region.drop_duplicates()
code_province_name_region.head()

,PlaceOfOccurrence,abbreviation,Region,Unidad Territorial
0,1,TO,Piemonte,Torino
312,2,VC,Piemonte,Vercelli
394,3,NO,Piemonte,Novara
481,4,CN,Piemonte,Cuneo
728,5,AT,Piemonte,Asti


In [56]:
# merge accident data with ISTAT data
data = pd.merge(df, code_province_name_region, left_on='Provincia', right_on='PlaceOfOccurrence', how='left')


In [57]:
data_TipoActividadEconómica = pd.read_csv("GestioneTariffaria.csv", sep = ";", encoding = "ISO-8859-1")


In [58]:
data_TipoActividadEconómica = pd.read_csv("GestioneTariffaria.csv", sep = ";", encoding = "ISO-8859-1")


In [59]:
data_TipoActividadEconómica.columns = ["TipoActividadEconómica", "DescrTipoActividadEconómica"]
data1 = pd.merge(data, data_TipoActividadEconómica, on='TipoActividadEconómica', how='left')

In [60]:
data_DetalleActividadEconómica = pd.read_csv("SettoreAttivitaEconomica.csv", sep = ";", encoding = "ISO-8859-1")

In [61]:
data2 = pd.merge(data1, data_DetalleActividadEconómica, left_on='DetalleActividadEconómica', right_on='SettoreAttivitaEconomica', how='left')

In [62]:
data_SectorEconómico = pd.read_csv("GrandeGruppoTariffario.csv", sep = ";", encoding = "ISO-8859-1")

In [63]:
data_SectorEconómico.columns = ["SectorEconómico", "DescrSectorEconómico"]
data3 = pd.merge(data2, data_SectorEconómico, on='SectorEconómico', how='left')

In [64]:
conteo_SectorX = data3['DescrSectorEconómico'].value_counts()

# Imprimir el resultado
print('Número de veces que aparece cada valor único en DescrSectorEconómico:')
print(conteo_SectorX)

Número de veces que aparece cada valor único en DescrSectorEconómico:
DescrSectorEconómico
GG0 Attività varie              122148
X Non determinato                61187
GG3 Costruzioni e impianti       43869
GG6 Metalli e macchinari         35839
GG9 Trasporti e magazzini        26028
GG1 Lav. agricole e alimenti     10565
GG2 Chimica, carta e cuoi         8747
GG5 Legno e affini                5617
GG7 Mineraria, rocce e vetro      4445
GG8 Tessile e confezioni          4328
GG4 Energia e comunicazioni       1345
Name: count, dtype: int64


In [65]:
data3 = data3[data3['DescrSectorEconómico'] != 'X Non determinato']
data3.shape

(262931, 40)

In [66]:
data3 = data3.fillna(0)

In [67]:
data3['Muerte'] = data3['FechaMuerte'].apply(lambda fecha: 1 if fecha != 0 else 0)

In [68]:
DescrTipoActividadEconómica_ = {'Non determinata': 'No Determinada',
                           'Industria':'Industria',
                           'Altre Attività':'Otra Actividad',
                           'Artigianato': 'Artesanía',
                           'Terziario':'Terciario'}
                           
data3['DescrTipoActividadEconómica']=data3['DescrTipoActividadEconómica'].map(DescrTipoActividadEconómica_)

In [69]:
# Diccionario de mapeo para 'SectorEconómico'
DescrSectorEconómico_ = {'GG9 Trasporti e magazzini': 'Transporte y almacenamiento', 
                         'GG0 Attività varie': 'Actividades diversas', 'GG6 Metalli e macchinari': 'Metales y maquinaria', 
                         'GG3 Costruzioni e impianti': 'Construcción e instalaciones', 'GG1 Lav. agricole e alimenti': 'Agricultura y alimentación', 
                         'GG8 Tessile e confezioni': 'Textil y confección', 'GG2 Chimica, carta e cuoi': 'Química, papel y cuero', 
                         'GG5 Legno e affini': 'Madera y afines', 'GG7 Mineraria, rocce e vetro': 'Minería, rocas y vidrio', 
                         'GG4 Energia e comunicazioni': 'Energía y comunicaciones'}

# Aplicar el mapeo a la columna 'SectorEconómico'
data3['DescrSectorEconómico'] = data3['DescrSectorEconómico'].map(DescrSectorEconómico_)

In [70]:
Gestión_ = {'I':1,
           'A':2,
           'S':3}
data3['Gestión']=data3['Gestión'].map(Gestión_)

In [71]:
TipoActividadEconómica_ = {'ND':0,
                           '1':1,
                           '2':2,
                           '3':3,
                           '4':4}
                           
data3['TipoActividadEconómica']=data3['TipoActividadEconómica'].map(TipoActividadEconómica_)

In [72]:
# Diccionario de mapeo para 'SectorEconómico'
SectorEconómico_ = {'ND': 10, '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}

# Aplicar el mapeo a la columna 'SectorEconómico'
data3['SectorEconómico'] = data3['SectorEconómico'].map(SectorEconómico_)

In [73]:
# Crear un diccionario que mapea los intervalos a las clases definidas en el texto
intervalos = {
    (1, 5): 1,
    (6, 15): 2,
    (16, 25): 3,
    (26, 50): 4,
    (51, 85): 5,
    (86, 100): 6
}

data3['GradoDeterioro'] = pd.cut(data3['GradoDeterioro'], bins=[0, 5, 15, 25, 50, 85, 100], labels=list(intervalos.values()), include_lowest=True)
# Transformar la columna 'GradoDeterioro' a tipo int
data3['GradoDeterioro'] = data3['GradoDeterioro'].astype(int)

In [74]:
# Crear un diccionario que mapea los intervalos a las clases definidas en el texto
intervalos = {
    (16, 25): 1,
    (26, 35): 2,
    (36, 45): 3,
    (46, 55): 4,
    (56, 100): 5  # Ajusté el límite superior a 56 para que coincida con el diccionario
}

data3['Edad'] = pd.cut(data3['Edad'], bins=[16, 26, 36, 46, 56, 100], labels=list(intervalos.values()), include_lowest=True)

# Transformar la columna 'Edad' a tipo int
data3['Edad'] = data3['Edad'].astype(int)


In [75]:
# Seleccionar las columnas que deseas conservar
columns_to_keep = ['Género', 'Edad', 'GradoDeterioro', 
                   'SectorEconómico', 'Año', 'Region']

# Crear un nuevo DataFrame solo con las columnas seleccionadas
data = data3[columns_to_keep]

In [76]:
Género_dummy = pd.get_dummies(data['Género'], prefix='Género')

Género_dummy = Género_dummy.astype(int)

# Concatenar las nuevas columnas dummy al DataFrame original
data = pd.concat([data, Género_dummy], axis=1)


In [77]:
data=data.drop(['Género', 'Género_F'], axis=1)


In [78]:
regiones = {'Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna', 'Friuli-Venezia Giulia', 'Lazio', 
            'Liguria', 'Lombardia', 'Marche', 'Molise', 'Piemonte', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana', 
            'Trentino-Alto Adige/Südtirol', 'Umbria', "Valle d'Aosta/Vallée d'Aoste", 'Veneto'}

divisiones = {
    'Noroccidental': {'Liguria', 'Lombardia', 'Piemonte', "Valle d'Aosta/Vallée d'Aoste"},
    'Nororiental': {'Emilia-Romagna', 'Friuli-Venezia Giulia', 'Trentino-Alto Adige/Südtirol', 'Veneto'},
    'Central': {'Lazio', 'Marche', 'Toscana', 'Umbria'},
    'Meridional': {'Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Molise', 'Puglia'},
    'Insular': {'Sardegna', 'Sicilia'}
}

# Mapear cada región a su división correspondiente
mapeo_divisiones = {}
for region in regiones:
    for division, regiones_division in divisiones.items():
        if region in regiones_division:
            mapeo_divisiones[region] = division
            break

data['Region'] = data['Region'].map(mapeo_divisiones)

In [79]:
Region_ = {'Noroccidental':4, 
           'Nororiental':3, 
           'Central':2, 
           'Meridional':0,
           'Insular':1}
                           
data['Region']=data['Region'].map(Region_)
data.head()

,Edad,GradoDeterioro,SectorEconómico,Año,Region,Género_M
0,4,1,3,2018,0,1
1,5,2,0,2018,0,0
2,5,2,0,2021,0,1
5,5,3,3,2022,0,1
8,5,1,0,2021,0,0


In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 262931 entries, 0 to 324117
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   Edad             262931 non-null  int32
 1   GradoDeterioro   262931 non-null  int32
 2   SectorEconómico  262931 non-null  int64
 3   Año              262931 non-null  int32
 4   Region           262931 non-null  int64
 5   Género_M         262931 non-null  int32
dtypes: int32(4), int64(2)
memory usage: 10.0 MB


In [81]:
data4 = data.sample(n=10000, random_state=42)

In [82]:
from sklearn.preprocessing import StandardScaler

# Selecciona las columnas que deseas estandarizar
columnas_a_estandarizar = ['Género_M', 'Edad', 'GradoDeterioro', 'SectorEconómico', 'Año', 'Region']  # Reemplaza con los nombres de las columnas que desees estandarizar

scaler = StandardScaler()

data4_scaled = data4.copy()  # Crea una copia del DataFrame original
data4_scaled[columnas_a_estandarizar] = scaler.fit_transform(data4[columnas_a_estandarizar])

# Top 20 Silhouette

In [83]:
from sklearn.metrics import silhouette_score
from itertools import combinations

# Obtener todas las combinaciones posibles de 4 a 6 variables
all_variable_combinations = []
for r in range(4, 6):
    combinations_r = list(combinations(data4_scaled.columns, r))
    all_variable_combinations.extend(combinations_r)

# Crear un diccionario para almacenar los resultados de cada combinación
results = {}

# Crear un diccionario para rastrear el número de veces que se ha utilizado cada número de clústeres
cluster_count = {}

# Iterar sobre las combinaciones y ejecutar kMeans
for i, variable_combination in enumerate(all_variable_combinations):
    # Seleccionar solo las columnas necesarias del DataFrame
    subset_data = data4_scaled[list(variable_combination)]

    # Aplicar el algoritmo kMeans con un número variable de clústeres (4 a 15)
    for num_clusters in range(3, 16):
        
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        distances = kmeans.fit_transform(subset_data)

        # Obtener los clústeres a los que pertenece cada punto
        clusters = kmeans.labels_

        # Calcular el índice de silhouette
        silhouette_avg = silhouette_score(subset_data, clusters)

        # Calcular el promedio por clúster
        cluster_means = pd.DataFrame(subset_data)
        cluster_means['Cluster'] = clusters
        cluster_means = cluster_means.groupby('Cluster').mean()

        # Almacenar los resultados en el diccionario
        key = (hash(variable_combination), num_clusters)
        results[key] = (cluster_means, silhouette_avg)

        # Incrementar el contador para el número de clústeres utilizado
        cluster_count[num_clusters] = cluster_count.get(num_clusters, 0) + 1

# Ordenar los resultados por el índice de silhouette
sorted_results = sorted(results.items(), key=lambda x: x[1][1], reverse=True)

# Imprimir las 10 mejores combinaciones con variables únicas
selected_combinations = set()
top_20_results = []

for result in sorted_results:
    (variable_combination, num_clusters), (cluster_means, silhouette_avg) = result
    if variable_combination not in selected_combinations:
        selected_combinations.add(variable_combination)
        top_20_results.append(result)

    if len(top_20_results) == 20:
        break
# Imprimir las 20 mejores combinaciones
for i, (variable_combination, num_clusters) in enumerate(top_20_results):
    print(f"\nTop {i+1} Variable Combination: {variable_combination}, Number of Clusters: {num_clusters}")
    print(cluster_means.style.bar(axis=0))


Top 1 Variable Combination: (-2280429558856369004, 14), Number of Clusters: (         GradoDeterioro  SectorEconómico    Region  Género_M
Cluster                                                     
0             -0.215425        -0.816104 -1.452063 -1.680068
1              0.646383        -0.496110 -0.902251  0.595214
2             -0.766250        -0.433723  0.474158  0.595214
3              0.734248        -0.823597  0.447981 -1.680068
4              0.739781         1.442849  0.566670  0.595214
5             -0.766250        -0.508286 -1.472543  0.595214
6             -0.766250        -0.826559  1.181901 -1.680068
7              2.510267        -0.212655  0.488612  0.468489
8             -0.216609         1.448554 -1.487270  0.595214
9             -0.175373         1.544907  0.327437 -1.680068
10            -0.766250         1.393911  0.536229  0.595214
11             2.676704         0.681527 -1.166734  0.544652
12             0.569378        -0.377115  0.844311  0.595214
13     

# Top 20 Davies - Bouldin

In [41]:
from sklearn.metrics import davies_bouldin_score
from itertools import combinations

# Obtener todas las combinaciones posibles de 4 a 6 variables
all_variable_combinations = []
for r in range(4, 6):
    combinations_r = list(combinations(data4_scaled.columns, r))
    all_variable_combinations.extend(combinations_r)

# Crear un diccionario para almacenar los resultados de cada combinación
results = {}

# Crear un diccionario para rastrear el número de veces que se ha utilizado cada número de clústeres
cluster_count = {}

# Iterar sobre las combinaciones y ejecutar kMeans
for i, variable_combination in enumerate(all_variable_combinations):
    # Seleccionar solo las columnas necesarias del DataFrame
    subset_data = data4_scaled[list(variable_combination)]

    # Aplicar el algoritmo kMeans con un número variable de clústeres (4 a 15)
    for num_clusters in range(4, 16):
        
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        distances = kmeans.fit_transform(subset_data)

        # Obtener los clústeres a los que pertenece cada punto
        clusters = kmeans.labels_

        # Calcular el índice de Davies-Bouldin
        db_index = davies_bouldin_score(subset_data, clusters)

        # Calcular el promedio por clúster
        cluster_means = pd.DataFrame(subset_data)
        cluster_means['Cluster'] = clusters
        cluster_means = cluster_means.groupby('Cluster').mean()

        # Almacenar los resultados en el diccionario
        key = (hash(variable_combination), num_clusters)
        results[key] = (cluster_means, db_index)

        # Incrementar el contador para el número de clústeres utilizado
        cluster_count[num_clusters] = cluster_count.get(num_clusters, 0) + 1

# Ordenar los resultados por el índice de Davies-Bouldin (menor es mejor en este caso)
sorted_results = sorted(results.items(), key=lambda x: x[1][1])

# Imprimir las 10 mejores combinaciones con variables únicas
selected_combinations = set()
top_20_results = []

for result in sorted_results:
    (variable_combination, num_clusters), (cluster_means, db_index) = result
    if variable_combination not in selected_combinations:
        selected_combinations.add(variable_combination)
        top_20_results.append(result)

    if len(top_20_results) == 20:
        break

# Imprimir las 20 mejores combinaciones
for i, (variable_combination, num_clusters) in enumerate(top_20_results):
    print(f"\nTop {i+1} Variable Combination: {variable_combination}, Number of Clusters: {num_clusters}, Davies-Bouldin Index: {db_index}")


Top 1 Variable Combination: (-2492953536713390277, 13), Number of Clusters: (         GradoDeterioro  SectorEconómico       Año  Género_M
Cluster                                                     
0              0.757588        -0.823406 -0.245933 -1.680068
1             -0.766250         1.420660  1.153285  0.595214
2             -0.766250        -0.444760  0.829015  0.595214
3              0.786393         1.051911 -1.065606  0.595214
4              0.761096        -0.622742 -0.618712  0.595214
5              3.276134         0.211135 -0.146420  0.398678
6             -0.766250        -0.834089 -0.650768 -1.680068
7             -0.766250         1.378146 -0.664811  0.595214
8              0.761673        -0.308888  0.973268  0.595214
9             -0.515015        -0.818036  1.230595 -1.680068
10            -0.766250        -0.477759 -0.967433  0.595214
11            -0.198155         1.536476 -0.042769 -1.680068
12             0.776592         1.481005  0.827168  0.595214, 0.8800

# Top 20 Calinski-Harabasz

In [42]:
from sklearn.metrics import calinski_harabasz_score
from itertools import combinations

# Suponiendo que 'data' es tu DataFrame y las variables están en 'variables_name'
# También suponiendo que tienes una columna 'Cluster' en tu DataFrame

# Obtener todas las combinaciones posibles de 4 a 6 variables 
all_variable_combinations = []
for r in range(4, 6):
    combinations_r = list(combinations(data4_scaled.columns, r))
    all_variable_combinations.extend(combinations_r)

# Crear un diccionario para almacenar los resultados de cada combinación
results = {}

# Crear un diccionario para rastrear el número de veces que se ha utilizado cada número de clústeres
cluster_count = {}

# Iterar sobre las combinaciones y ejecutar kMeans
for i, variable_combination in enumerate(all_variable_combinations):
    # Seleccionar solo las columnas necesarias del DataFrame
    subset_data = data4_scaled[list(variable_combination)]

    # Aplicar el algoritmo kMeans con un número variable de clústeres (4 a 15)
    for num_clusters in range(4, 16):
        
        kmeans = KMeans(n_clusters=num_clusters, random_state=42)
        distances = kmeans.fit_transform(subset_data)

        # Obtener los clústeres a los que pertenece cada punto
        clusters = kmeans.labels_

        # Calcular el índice de Calinski-Harabasz
        calinski_score = calinski_harabasz_score(subset_data, clusters)

        # Calcular el promedio por clúster
        cluster_means = pd.DataFrame(subset_data)
        cluster_means['Cluster'] = clusters
        cluster_means = cluster_means.groupby('Cluster').mean()

        # Almacenar los resultados en el diccionario
        key = (hash(variable_combination), num_clusters)
        results[key] = (cluster_means, calinski_score)

        # Incrementar el contador para el número de clústeres utilizado
        cluster_count[num_clusters] = cluster_count.get(num_clusters, 0) + 1

# Ordenar los resultados por el índice de Calinski-Harabasz
sorted_results = sorted(results.items(), key=lambda x: x[1][1], reverse=True)

# Imprimir las 10 mejores combinaciones con variables únicas
selected_combinations = set()
top_20_results = []

for result in sorted_results:
    (variable_combination, num_clusters), (cluster_means, calinski_score) = result
    if variable_combination not in selected_combinations:
        selected_combinations.add(variable_combination)
        top_20_results.append(result)

    if len(top_20_results) == 20:
        break

# Imprimir las 20 mejores combinaciones
for i, (variable_combination, num_clusters) in enumerate(top_20_results):
    print(f"\nTop {i+1} Variable Combination: {variable_combination}, Number of Clusters: {num_clusters}")
    print(f"Calinski-Harabasz Score: {calinski_score}")


Top 1 Variable Combination: (-2280429558856369004, 5), Number of Clusters: (         GradoDeterioro  SectorEconómico    Region  Género_M
Cluster                                                     
0             -0.185732        -0.765794  0.057993 -1.680068
1             -0.190105        -0.420917  0.476083  0.595214
2             -0.220095        -0.221120 -1.471609  0.595214
3             -0.255285         1.483285  0.340553  0.463671
4              2.423486         0.248223 -0.108217  0.480878, 4391.108915398963)
Calinski-Harabasz Score: 2277.6777331021112

Top 2 Variable Combination: (-2492953536713390277, 7), Number of Clusters: (         GradoDeterioro  SectorEconómico       Año  Género_M
Cluster                                                     
0             -0.218512         1.424351 -0.981042  0.462204
1             -0.242723         1.461914  0.828318  0.500549
2             -0.196020        -0.429042  0.798801  0.595214
3             -0.189222        -0.733525  1.144438